<center>
<img src="https://2223moodle.isel.pt/pluginfile.php/1/theme_adaptable/logo/1631635830/logo-isel_inv3.png" width="250">
<h3>Licenciatura em Engenharia Informática e Multimédia</h3><br>
<br>
<h2>Processamento de Imagem e Visão (PIV)</h3>
<h3>2º Trabalho Laboratorial – Estimação e classificação de movimento </h3>
<br>

<br>
Trabalho Realizado por:<br>
Gonçalo Silva <b>A48328</b><br>
Diogo Lobo <b>A48168</b><br>
Turma 52D<br><br>
Docente: João Pedro Costa <br>
<br>
27 de Novembro 2024
</center>


In [2]:
# Imports
import cv2
import numpy as np
import time
import pygame
from pygame.locals import QUIT
from pygame import surfarray

In [3]:
#Inicialização dos tamanhos da janela e objetos
window_width = 1280
window_height = 700
cube_size = 50
cube_x = window_width // 4 - cube_size
cube_y = window_height // 2 - cube_size
cube_color = (0, 0, 255)

## Final 

In [4]:
import cv2
import numpy as np
import pygame

# Inicializa a câmera e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack")
fonte = pygame.font.SysFont("Arial", 64)

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para o espaço HSV e escala de cinza
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Ajuste conforme necessário
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    
    # mask = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=2)
    # mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=2)

    # Encontra contornos da máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Inicializa flag para verificar se a mão foi detectada
    hand_detected = False

    if contours:
        # Seleciona o maior contorno, assumindo que é a mão
        largest_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(largest_contour) > 5000:  # Filtra áreas pequenas (ajustar conforme necessário)
            hand_detected = True

            # Desenha o contorno da mão
            cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)

            # Cria uma máscara apenas para a mão detectada
            hand_mask = np.zeros_like(mask)
            cv2.drawContours(hand_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

            # Aplica a máscara da mão à imagem em escala de cinza
            hand_region = cv2.bitwise_and(gray, gray, mask=hand_mask)

            # Detecta características usando goodFeaturesToTrack apenas na região da mão
            corners = cv2.goodFeaturesToTrack(hand_region, maxCorners=50, qualityLevel=0.01, minDistance=10)
            if corners is not None:
                for corner in corners:
                    x, y = corner.ravel()
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)  # Desenha círculos nos pontos detectados

    ##### Vizualização da câmera no Pygame e sua colocação #####

    # Converte de BGR (OpenCV) para RGB (Pygame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.rot90(frame)  # Pygame espera a imagem transposta
    pygame_surface = pygame.surfarray.make_surface(frame)
    window.fill((0, 0, 0))  # Limpa a tela antes de atualizar o frame
    window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

    # Mostra o texto apenas se a mão for detectada
    if hand_detected:
        texto_surface = fonte.render("Hand Detected", True, (255, 0, 0))  # Texto em vermelho
        window.blit(texto_surface, (window_width - 520, 10))  # Posição do texto

    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()


## Teste 1

In [5]:
#Inicialização da Camera 
camera = 0
cam = cv2.VideoCapture(camera)

running = True


pygame.init()

window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Webcam in Pygame")

running = True
fonte = pygame.font.SysFont("Arial", 32)
previousFrame = None
thr = 10
kernelSize = 9
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernelSize,kernelSize),(-1,-1))

if cam.isOpened():
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        flag, frame = cam.read()
        if flag:

            frameE = cv2.medianBlur(frame,kernelSize)

            if previousFrame is None:
                previousFrame = frame

            diff_image = cv2.absdiff(frame,previousFrame)
            _ ,bw1 = cv2.threshold(diff_image,thr,255,cv2.THRESH_BINARY)
            _, bw2 = cv2.threshold(np.uint8(bw1.sum(axis=2)),1,255,cv2.THRESH_BINARY)

            erode_image = cv2.erode(bw2,kernel)
            dilate_image = cv2.dilate(erode_image,kernel,iterations=2)

            (num_labels,label_image,stats,centroids) = cv2.connectedComponentsWithStats(dilate_image)

            previousFrame = frame.copy()

            motionFlag = False

            for i in range(1,num_labels):
                if(stats[i,4]>0.05*bw2.shape[0]*bw2.shape[1]):
                    cv2.rectangle(frame,(stats[i,0],stats[i,1]),(stats[i,0]+stats[i,2],stats[i,1]+stats[i,3]),(0,0,255),thickness=2)
                    motionFlag = True

            if motionFlag:
                texto_surface = fonte.render("Motion Detected", True, (255, 0, 0))  # Texto em vermelho
                window.blit(texto_surface, (10, 10))  # Posição do texto

            ##### Vizualização da camera no Pygame e sua colocação ######

            # Converte de BGR (OpenCV) para RGB (Pygame)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = np.rot90(frame)  # Pygame espera a imagem transposta
            pygame_surface = pygame.surfarray.make_surface(frame)
            window.blit(pygame_surface, (window_width // 2, window_height // 7))  # Centraliza na tela

        pygame.display.update()  # Atualiza a tela

    # Libera recursos ao sair
    cam.release()
    pygame.quit()

else:
    print("Error: Could not open Camera")


## Teste 2 

In [6]:
import cv2
import numpy as np
import pygame

# Inicializa a câmera e Pygame
pygame.init()
cap = cv2.VideoCapture(0)
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Hand Detection with GoodFeaturesToTrack1")

running = True

while running:
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Segmentação baseada na cor da pele (opcional)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Aplica a máscara e detecta as características
    masked_gray = cv2.bitwise_and(gray, gray, mask=mask)

    # Detecta características usando goodFeaturesToTrack
    corners = cv2.goodFeaturesToTrack(masked_gray, maxCorners=50, qualityLevel=0.01, minDistance=10)
    if corners is not None:
        for corner in corners:
            x, y = corner.ravel()
            cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Desenha círculos nos pontos detectados

    # Exibe a imagem com os pontos detectados no Pygame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = np.rot90(frame_rgb)
    pygame_surface = pygame.surfarray.make_surface(frame_rgb)
    window.blit(pygame_surface, (0, 0))
    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()
